In [1]:
import getpass
import os
username = os.environ['USER']
if os.system('klist | grep Default | grep ' + username + '@CERN.CH'):
    os.system('echo %s' % getpass.getpass() + " | kinit " + username)

········


In [2]:
import ROOT

Welcome to JupyROOT 6.24/00


In [3]:
file = ROOT.TFile.Open("root://eosproject//eos/project/h/highlo/workshop/ZS_ZC/messages/20150504_20150508.root")

In [4]:
file.ls()

TNetXNGFile**		root://eosproject//eos/project/h/highlo/workshop/ZS_ZC/messages/20150504_20150508.root	
 TNetXNGFile*		root://eosproject//eos/project/h/highlo/workshop/ZS_ZC/messages/20150504_20150508.root	
  KEY: TTree	LOB;62	LOB [current cycle]
  KEY: TTree	LOB;61	LOB [backup cycle]


In [5]:
ROOT.EnableImplicitMT()
rdf = ROOT.RDataFrame("LOB", file)

Warning in <RTaskArenaWrapper>: CPU Bandwith Control Active. Proceeding with 2 threads accordingly


In [6]:
c = ROOT.TCanvas("myCanvasName","The Canvas Title",800,600)

In [7]:
rdfPrice = rdf.Filter('ZCN5_IsOriginalMessage == true')\
  .Define('AskCent', 'ZCN5_AskPrice1 * ZCN5_TickSize')\
  .Define('BidCent', 'ZCN5_BidPrice1 * ZCN5_TickSize')
histBid = rdfPrice.Filter('ActionSide == 0')\
  .Histo1D(('hAskPrizeBid','bid price in cent', 40, 340, 380), 'BidCent')
histAsk = rdfPrice.Filter('ActionSide == 1')\
  .Histo1D(('hAskPrizeAsk','ask price in cent', 40, 340, 380), 'AskCent')


In [8]:
%jsroot on
histBid.Draw()
histAsk.SetLineColor(ROOT.kRed)
histAsk.Draw("SAME")
c.Draw()